In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as tf_keras

In [ ]:
# 데이터 준비
(X_train, y_train), (X_test, y_test) = tf_keras.datasets.reuters.load_data(num_words=10000)

In [ ]:
# imdb 단어 사전 조회
word_idx_map = tf_keras.datasets.imdb.get_word_index()

In [ ]:
# 단어 : 인덱스 조합 -> 인덱스 : 단어 조합으로 변경
idx_word_map = { idx : word for (word, idx) in word_idx_map.items() }

In [ ]:
# 입력 데이터 구성 : 각 문장을 (1, 10000) 배열로 변환
def vectorize_setences(sentences, dimension=10000, bow=True):
  result = np.zeros(shape=(len(sentences), dimension)) # 0으로 초기화한 (문장갯수, 단어갯수)의 2차원 배열 구성
  for i, sentence in enumerate(sentences):
    for word_idx in sentence:
      if bow:
        result[i, word_idx] += 1 # 단어 빈도
      else:
        result[i, word_idx] = 1 # 단어 발생

  return result

In [ ]:
# 입력 데이터 변환
X_train2 = vectorize_setences(X_train, bow=False)
X_test2 = vectorize_setences(X_test, bow=False)

In [ ]:
np.unique(y_train).shape

(46,)

In [ ]:
# 모델 만들기 1
model = tf_keras.Sequential()
model.add(tf_keras.layers.Input(shape=(10000, )))
model.add(tf_keras.layers.Dense(units=512, activation='relu', kernel_regularizer=tf_keras.regularizers.L2(l2=0.001)))
model.add(tf_keras.layers.Dense(units=256, activation='relu', kernel_regularizer=tf_keras.regularizers.L2(l2=0.001)))
model.add(tf_keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf_keras.regularizers.L2(l2=0.001)))
model.add(tf_keras.layers.Dense(units=46, activation="softmax"))

In [ ]:
# 모델 만들기 2
model2 = tf_keras.Sequential()
model2.add(tf_keras.layers.Input(shape=(10000, )))
model2.add(tf_keras.layers.Dense(units=512, activation='relu'))
model2.add(tf_keras.layers.Dropout(rate=0.5, seed=42))
model2.add(tf_keras.layers.Dense(units=256, activation='relu'))
model2.add(tf_keras.layers.Dropout(rate=0.5, seed=42))
model2.add(tf_keras.layers.Dense(units=128, activation='relu'))
model2.add(tf_keras.layers.Dropout(rate=0.5, seed=42))
model2.add(tf_keras.layers.Dense(units=46, activation="softmax"))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 512)               5120512   
                                                                 
 dense_9 (Dense)             (None, 256)               131328    
                                                                 
 dense_10 (Dense)            (None, 128)               32896     
                                                                 
 dense_11 (Dense)            (None, 46)                5934      
                                                                 
Total params: 5290670 (20.18 MB)
Trainable params: 5290670 (20.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 모델 만들기 3
model3 = tf_keras.Sequential()
model3.add(tf_keras.layers.Input(shape=(10000, )))
model3.add(tf_keras.layers.Dense(units=512, activation='relu', kernel_initializer='he_uniform'))
# model3.add(tf_keras.layers.Dense(units=256, activation='relu', kernel_initializer='he_normal'))
model3.add(tf_keras.layers.Dense(units=256, activation='relu', kernel_initializer=tf_keras.initializers.HeNormal(seed=24)))
model3.add(tf_keras.layers.Dense(units=128, activation='relu', kernel_initializer='he_uniform'))
model3.add(tf_keras.layers.Dense(units=46, activation="softmax"))

In [ ]:
# 모델 만들기 4
model4 = tf_keras.Sequential()
model4.add(tf_keras.layers.Input(shape=(10000, )))

model4.add(tf_keras.layers.Dense(units=512))
model4.add(tf_keras.layers.BatchNormalization())
model4.add(tf_keras.layers.Activation('relu'))

model4.add(tf_keras.layers.Dense(units=256))
model4.add(tf_keras.layers.BatchNormalization())
model4.add(tf_keras.layers.Activation('relu'))

model4.add(tf_keras.layers.Dense(units=128))
model4.add(tf_keras.layers.BatchNormalization())
model4.add(tf_keras.layers.Activation('relu'))

model4.add(tf_keras.layers.Dense(units=46, activation="softmax"))

In [ ]:
model4.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
history = model4.fit(X_train2, y_train, batch_size=256, epochs=10, validation_split=0.2)

Epoch 1/10
29/29 [==============================] - 8s 210ms/step - loss: 1.7224 - accuracy: 0.6430 - val_loss: 2.8975 - val_accuracy: 0.7435
Epoch 2/10
29/29 [==============================] - 6s 222ms/step - loss: 0.5330 - accuracy: 0.8955 - val_loss: 2.3083 - val_accuracy: 0.7095
Epoch 3/10
29/29 [==============================] - 5s 189ms/step - loss: 0.2486 - accuracy: 0.9520 - val_loss: 1.8285 - val_accuracy: 0.6461
Epoch 4/10
29/29 [==============================] - 6s 223ms/step - loss: 0.1626 - accuracy: 0.9609 - val_loss: 1.5094 - val_accuracy: 0.6544
Epoch 5/10
29/29 [==============================] - 6s 189ms/step - loss: 0.1251 - accuracy: 0.9624 - val_loss: 1.2842 - val_accuracy: 0.6845
Epoch 6/10
29/29 [==============================] - 5s 175ms/step - loss: 0.1072 - accuracy: 0.9627 - val_loss: 1.0909 - val_accuracy: 0.7446
Epoch 7/10
29/29 [==============================] - 6s 206ms/step - loss: 0.0998 - accuracy: 0.9617 - val_loss: 1.0451 - val_accuracy: 0.7457
Epoch 